In [1]:
from google.colab import drive 

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls "/content/drive/MyDrive/AIAssignment2/"

Ass2w12.ipynb  Ass2w3.ipynb  Ass2w4.ipynb  imdb  movie.csv  Untitled0.ipynb


In [9]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/AIAssignment2/movie.csv')

In [11]:
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [13]:
df = df[['Title', 'Genre', 'Director', 'Actors', 'Plot', 'imdbRating']]

In [21]:
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [22]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = nltk.word_tokenize(text)
    words = [word for word in words if not word in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)


In [23]:
df['Genre'] = df['Genre'].apply(preprocess_text)
df['Director'] = df['Director'].apply(preprocess_text)
df['Actors'] = df['Actors'].apply(preprocess_text)
df['Plot'] = df['Plot'].apply(preprocess_text)


In [24]:
df['Bag-of-Words'] = df['Title'] + ' ' + df['Genre'] + ' ' + df['Director'] + ' ' + df['Actors'] + ' ' + df['Plot']

In [25]:
!pip install gensim transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.5 MB/s eta 0:00:00


In [26]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModel

In [27]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(df['Bag-of-Words'])
tfidf_embeddings = np.asarray(tfidf.mean(axis=1))

In [29]:
w2v_model = Word2Vec(df['Bag-of-Words'].apply(lambda x: x.split()), min_count=1, vector_size=300)
w2v_embeddings = np.asarray([np.mean([w2v_model.wv[word] for word in doc.split() if word in w2v_model.wv], axis=0) for doc in df['Bag-of-Words']])


In [30]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModel.from_pretrained('bert-base-cased')
bert_embeddings = []

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
for text in df['Bag-of-Words']:
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    outputs = model(**inputs).pooler_output.detach().numpy()
    bert_embeddings.append(outputs)
bert_embeddings = np.asarray(bert_embeddings)

In [37]:
bert_embeddings_2d = bert_embeddings.reshape(bert_embeddings.shape[0], -1)

tfidf_sim = cosine_similarity(tfidf_embeddings)
w2v_sim = cosine_similarity(w2v_embeddings)
bert_sim = cosine_similarity(bert_embeddings_2d)

similarity_matrix = (tfidf_sim + w2v_sim + bert_sim) / 3
print(similarity_matrix)

[[1.00000008 0.76480095 0.75795051 ... 0.74008522 0.74539584 0.7307552 ]
 [0.76480095 0.99999992 0.84670399 ... 0.71093289 0.76582071 0.72549134]
 [0.75795051 0.84670399 1.         ... 0.76118485 0.80093804 0.73271878]
 ...
 [0.74008522 0.71093289 0.76118485 ... 1.         0.71593536 0.73845694]
 [0.74539584 0.76582071 0.80093804 ... 0.71593536 0.99999998 0.71842502]
 [0.7307552  0.72549134 0.73271878 ... 0.73845694 0.71842502 1.00000004]]


In [36]:
indices = pd.Series(df.index, index=df['Title'])

idx = indices['The Dark Knight']

sim_scores = list(enumerate(similarity_matrix[idx]))

sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

top_similar_movies = [i[0] for i in sim_scores[1:11]]

print("Top 10 recommended movies for 'The Dark Knight':\n")
print(df['Title'].iloc[top_similar_movies])


Top 10 recommended movies for 'The Dark Knight':

62                                 The Dark Knight Rises
236                                                 Jaws
35                                        The Green Mile
47                                          The Prestige
2                                 The Godfather: Part II
81                                    Lawrence of Arabia
66                           Once Upon a Time in America
10     The Lord of the Rings: The Fellowship of the Ring
243                                The Battle of Algiers
177                                    Kill Bill: Vol. 1
Name: Title, dtype: object
